# **Experiment 1: Value Model Measurements**

1. **Workload Latencies**: Measure the response times of the system.
2. **Latency Prediction Q-Errors**: Evaluate the accuracy of the system's latency prediction model by calculating Q-Errors, which quantify the deviation between predicted and actual latencies.
3. **Inference Times**: Analyze the time taken for the system to process and generate predictions, providing insights into its computational efficiency.

### **Hardware Used**

| **System**                                      | **CPU**                                       | **RAM**                                      | **Storage**                                   | **GPU**                                          | **Platform**                                     |
|-------------------------------------------------|-----------------------------------------------|----------------------------------------------|-----------------------------------------------|-------------------------------------------------|-------------------------------------------------|
| **BAO**                                         | Intel Xeon Gold 6230                         | 15GB (VM) + 256GB (CPU)                      | Not specified                                 | Tesla T4 GPU                                     | Google Cloud Platform (N1-4 VM)                 |
| **LOGER**                                       | 2× Intel Xeon Gold                           | 256GB                                        | Not specified                                 | NVIDIA RTX 3090                                  | Physical Server                                 |
| **BALSA**                                       | Microsoft Azure VMs (8 cores)                | 64GB                                         | SSD                                           | NVIDIA Tesla M60 GPU                             | Microsoft Azure                                 |
| **FASTgres**                                    | Intel Xeon Gold 6216 (12 cores)              | 92GB                                         | 1.8 TiB storage                               | Not specified                                    | Physical Server                                 |
| **train-server (Hosts PostgreSQL)**             | QEMU Virtual CPU version 2.5                 | 110GB                                        | 2.0 TiB SSD                                   | None                                            | Physical Server                                 |
| **train-gpu-server (Hosts the optimizers)**     | Intel(R) Xeon(R) Gold 5318Y                  | 377GB                                        | 1.4 TiB SSD                                   | NVIDIA RTX A6000                                  | Physical Server                                 |

### **PostgreSQL Configurations**

| **PostgreSQL Config Parameter**  | **Description** | **Default Values** | **JOB** | **Bao** | **Balsa** | **LOGER** | **Our Framework** |
|----------------------------------|----------------|------------------|-------------|-------------|-----------------------|-------------|----------------|
| **Join Order** |  |  |  |  |  |  |  |
| geqo_threshold | Sets the threshold for Genetic Query Optimizer (GEQO) to be used | 12 | 18 |  |  | 2 or 1,024 |  |
| geqo | Enables or disables GEQO, which helps optimize complex queries | on |  |  | off | off | off |
| **Working Memory** |  |  |  |  |  |  |  |
| work_mem | Memory allocated for each query operation (e.g., sorting, hashing) | 4 MB | 2 GB |  | 4 GB | | 4 GB |
| shared_buffers | Memory used for PostgreSQL's buffer cache to reduce disk I/O | 128 MB | 4 GB | 4 GB | 32 GB | 64 GB | 32 GB |
| temp_buffers | Memory allocated for temporary tables during a session | 8 MB |  |  | 32 GB |  | 32 GB |
| effective_cache_size | Estimated memory available for caching disk pages | 4 GB | 32 GB |  |  |  | 64 GB |
| **Parallelization** |  |  |  |  |  |  |  |
| max_parallel_workers | Maximum number of parallel workers allowed | 8 |  |  |  | 1 | 8 |
| max_parallel_workers_per_gather | Maximum parallel workers per `Gather` node in a query plan | 8 |  |  |  | 1 | 8 |
| max_worker_processes | Maximum background worker processes allowed | 2 |  |  | 8 |  | 8 |
| **Scan Types** |  |  |  |  |  |  |  |
| enable_bitmapscan | Enables bitmap scans for efficient index-based retrieval | on |  | | off |  | on |
| enable_tidscan | Enables TID scans, which use tuple IDs for direct row lookups | on |  |  | off |  | on |


## **Benchmarks Used**

### **1. JOB and its Variants**

#### **Join Order Benchmark (Default)**

- Evaluates the efficiency of database query optimizers, focusing on join order optimization and cardinality estimation accuracy.
- 21 Tables, 6 Columns per Table on Average, 3.6 GB in size, 113 Queries in total, 8 average Joins per query

## **2. TPC-H and Its Variants**  

### **TPC-H-NO-NESTED-QUERIES**  
- Uses **templates 3, 5, 12, and 14** for training and **template 10** for testing, with **10 queries per template**.  
- Excludes queries containing **nested subqueries** in either the **FROM** or **WHERE** clause.  
- Templates **7, 8, and 13** are excluded due to their reliance on these nested structures, which **LOGER** and **FASTgres** cannot process.  
- **Example of Banned Queries:**  

  **1. Nested FROM Clause (Disallowed)**  
  ```sql
  SELECT o_orderkey, revenue
  FROM (
      SELECT l_orderkey AS o_orderkey, SUM(l_extendedprice) AS revenue
      FROM lineitem
      GROUP BY l_orderkey
  ) AS subquery
  WHERE revenue > 100000;
  ```
  - This query is excluded because it **uses a subquery in the FROM clause (nested FROM)**.  

  **2. Nested WHERE Clause (Disallowed)**  
  ```sql
  SELECT l_orderkey
  FROM lineitem
  WHERE l_extendedprice > (
      SELECT AVG(l_extendedprice)
      FROM lineitem
  );
  ```
  - This query is excluded because it **contains a subquery in the WHERE clause**.  

- **Compatible with:**  
  - BAO  
  - Balsa  
  - FASTgres  
  - LOGER  

### **3. TPC-DS**  

### **3.1 TPC-DS-SIMPLE** (originates from *LOGER*)
- Only queries in SPJ Format
- Uses **15 templates for training** and **5 templates for testing**, ensuring that all tables in the testing workload appear in the training workload.  
- Generates **3 queries per template** for both training and testing.  
- **Training Templates:** 3, 7, 12, 20, 26, 37, 42, 43, 50, 55, 62, 84, 91, 96, 99  
- **Testing Templates:** 18, 27, 52, 82, 98  
- **Compatible with:**
  - BAO
  - LOGER  
  - FASTgres (WIP)  


In [48]:
import pandas as pd
import os
import json
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import re
from pathlib import Path
from tqdm import tqdm
import glob

OPTIMIZERS = ['BAO', 'LOGER', 'LERO', 'NEO']
# ==============================================================================
# 1. DATA COLLECTION
# ==============================================================================

def parse_checkpoint_name(name):
    epoch_match = re.search(r"epoch-?(\d+)", name)
    queries_match = re.search(r"queries-?(\d+)", name)
    loss_match = re.search(r"loss-?([-\d.e]+)", name)
    epoch = int(epoch_match.group(1)) if epoch_match else -1
    queries = int(queries_match.group(1)) if queries_match else -1
    loss = float(loss_match.group(1)) if loss_match else -1.0
    return epoch, queries, loss

def discover_checkpoints(models_base_dir, optimizer_name):
    """
    Discovers trained model checkpoints for a given optimizer.
    Keeps .pt, .pkl, and valid checkpoint directories.
    Excludes metadata, baselines, and final_model files.
    """
    checkpoint_list = []
    optimizer_path = Path(models_base_dir) / optimizer_name.upper()
    subdirs = ["epoch_checkpoints", "query_checkpoints", "loss_checkpoints"]

    unique_names = set()
    allowed_extensions = {".pt", ".pkl"}  # valid checkpoint files

    exclude_patterns = (
        "_metadata.json",
        "_metadata.txt",
        "_baseline.pkl",
    )

    for subdir in subdirs:
        full_subdir_path = optimizer_path / subdir
        if not full_subdir_path.is_dir():
            continue

        for item_path in full_subdir_path.iterdir():
            # --- case 1: valid file checkpoint ---
            if item_path.is_file():
                if item_path.suffix not in allowed_extensions:
                    continue
                if any(str(item_path).endswith(pattern) for pattern in exclude_patterns):
                    continue
                if item_path.stem.startswith("final_model"):
                    continue
                base_name = item_path.stem

            # --- case 2: valid directory checkpoint ---
            elif item_path.is_dir():
                # skip final_model dirs or metadata dirs
                if item_path.name.startswith("final_model"):
                    continue
                # heuristic: treat any directory without extension as checkpoint
                base_name = item_path.name

            else:
                continue

            # avoid duplicates
            if base_name not in unique_names:
                # print(f"Discovered checkpoint: {base_name}")
                # print(f"  Path: {item_path}")
                unique_names.add(base_name)
                checkpoint_list.append({
                    "name": base_name,
                    "type": subdir,
                    "path": str(item_path)
                })
    return checkpoint_list

In [49]:
def extract_execution_time(plan_json):
    try:
        if isinstance(plan_json, list) and isinstance(plan_json[-1], dict) and 'Execution Time' in plan_json[-1]:
            return plan_json[-1].get('Execution Time', 0.0)
        if isinstance(plan_json, list) and 'Execution Time' in plan_json[0]:
            return plan_json[0]['Execution Time']
        if isinstance(plan_json, list) and isinstance(plan_json[0], list) and 'Execution Time' in plan_json[-1][0][0]:
            return plan_json[0][0][0]['Execution Time']
        if isinstance(plan_json, dict) and 'Execution Time' in plan_json:
            return plan_json.get('Execution Time', 0.0)
    except (IndexError, KeyError, TypeError): pass
    return 0.0

def extract_planning_time(plan_json):
    try:
        if isinstance(plan_json, list) and isinstance(plan_json[-1], dict) and 'Planning Time' in plan_json[-1]:
            return plan_json[-1].get('Planning Time', 0.0)
        if isinstance(plan_json, list) and 'Planning Time' in plan_json[0]:
            return plan_json[0]['Planning Time']
        if isinstance(plan_json, list) and isinstance(plan_json[0], list) and 'Planning Time' in plan_json[-1][0][0]:
            return plan_json[0][0][0]['Planning Time']
        if isinstance(plan_json, dict) and 'Planning Time' in plan_json:
            return plan_json.get('Planning Time', 0.0)
    except (IndexError, KeyError, TypeError): pass
    return 0.0

def build_trajectory_df_from_files(discovered_checkpoints, results_base_dir, optimizer_name):
    rows = []
    for checkpoint in tqdm(discovered_checkpoints, desc=f"Processing {optimizer_name} Checkpoints"):
        name, type_dir = checkpoint['name'], checkpoint['type']
        pattern = Path(results_base_dir) / "*" / optimizer_name.upper() / type_dir / name / "**" / "*_plan.json"
        plan_files = glob.glob(str(pattern), recursive=True)
        if not plan_files: continue
        total_lat_ms, q_errors = 0, []
        for plan_path in plan_files:
            pred_lat_ms, exec_time_ms = np.nan, 0
            try:
                with open(plan_path, 'r') as f: plan_data = json.load(f)
                exec_time_ms = extract_execution_time(plan_data)
                planning_time_ms = extract_planning_time(plan_data)
                if exec_time_ms > 0: total_lat_ms += exec_time_ms
                if planning_time_ms > 0: total_lat_ms += planning_time_ms
                if optimizer_name.upper() == 'BAO':
                    if isinstance(plan_data, list) and len(plan_data) > 0 and 'Bao' in plan_data[0]:
                        pred_lat_ms = plan_data[0]['Bao'].get('Bao prediction', np.nan)
                else:
                    metrics_path = plan_path.replace('_plan.json', '_metrics.json')
                    if os.path.exists(metrics_path):
                        # print(f"Loading metrics from {metrics_path}")
                        with open(metrics_path, 'r') as f: metrics_data = json.load(f)
                        pred_lat_ms = metrics_data.get('predicted_latency_ms', np.nan)
                if exec_time_ms > 0 and not np.isnan(pred_lat_ms) and pred_lat_ms > 0:
                    if optimizer_name.upper() == 'NEO':
                        pred_lat_ms *= 10
                    q_errors.append(max(pred_lat_ms / exec_time_ms, exec_time_ms / pred_lat_ms))
            except Exception: continue
        if total_lat_ms == 0: continue
        rows.append({
            "optimizer": optimizer_name, "checkpoint_type": type_dir.replace("_checkpoints", ""),
            **dict(zip(["epoch", "queries_seen", "model_loss"], parse_checkpoint_name(name))),
            "cumulative_latency_s": total_lat_ms / 1000.0, "avg_q_error": np.mean(q_errors) if q_errors else np.nan
        })
    return pd.DataFrame(rows)   

def get_postgres_baseline(results_base_dir):
    # search_pattern = Path(results_base_dir) / "POSTGRES" / "**" / "*_plan.json"
    search_pattern = Path(results_base_dir) / "**" / "classic_qep.json"
    print(f"Searching for Postgres baseline plans in {search_pattern}")
    plan_files = glob.glob(str(search_pattern), recursive=True)
    if not plan_files: return None
    total_latency_ms = sum(extract_execution_time(json.load(open(p, 'r'))) for p in plan_files)
    return total_latency_ms / 1000.0 if total_latency_ms > 0 else None

In [50]:
def plot_optimizer_grid(df_long, optimizer_name, output_dir, baseline_latency=None, smooth_window=5):
    """
    Generates a master grid plot for a SINGLE optimizer based on the provided data.
    The structure of the grid is automatically determined by the metrics and policies in df_long.
    """
    print(f"\nGenerating custom grid plot for {optimizer_name}...")
    plt.style.use('seaborn-v0_8-whitegrid')

    # Determine grid structure from the data provided
    row_order = sorted(df_long["Metric"].unique())
    col_order = sorted(df_long["Training Policy"].unique())

    # --- Setup plot ---
    palette = sns.color_palette("Purples", 6)
    g = sns.relplot(
        data=df_long, x="Progress Value", y="Metric Value",
        row="Metric", col="Training Policy",
        row_order=row_order, col_order=col_order,
        kind="line", marker="s", dashes=False, color=palette[4], label="Raw",
        facet_kws={'sharey': False, 'sharex': False, 'margin_titles': True},
        height=4, aspect=1.5
    )

    # --- Customize each subplot ---
    for (metric, policy), ax in g.axes_dict.items():
        subset = df_long[(df_long["Metric"] == metric) & (df_long["Training Policy"] == policy)]
        if subset.empty:
            continue

        subset = subset.sort_values("Progress Value").reset_index(drop=True)

        # Smoothed line
        smooth_y = subset["Metric Value"].rolling(window=smooth_window, min_periods=1, center=True).mean()
        ax.plot(subset["Progress Value"], smooth_y,
                color=palette[2], lw=2.5, alpha=0.6, label="Smoothed", marker="D")

        # Grid and axis customization
        ax.grid(which="major", axis="both", linestyle=":", alpha=0.5)
        ax.set_yscale('log')
        if metric == 'Workload Latency (s)' and baseline_latency is not None and optimizer_name.upper() != "NEO":
            ax.axhline(baseline_latency, ls='--', color='red', lw=1.5, label='PostgreSQL Baseline')
        if policy == "Model Loss":
            ax.invert_xaxis()

    # --- Final Touches ---
    g.set_titles(col_template="{col_name}", row_template="{row_name}")
    g.fig.suptitle(f'Performance Trajectory for {optimizer_name.upper()}', fontsize=16, y=1.05)
    
    handles, labels = g.axes.flatten()[0].get_legend_handles_labels()
    unique_labels = dict(zip(labels, handles))
    g.fig.legend(unique_labels.values(), unique_labels.keys(),
                 loc="upper center", ncol=3, frameon=True, bbox_to_anchor=(0.5, 0.98))

    plt.subplots_adjust(top=0.85)

    filename = f"trajectory_grid_{optimizer_name}.pdf"
    plt.savefig(Path(output_dir) / filename, bbox_inches='tight')
    plt.show()
    print(f"Saved plot: {filename}")

In [51]:
import matplotlib.lines as mlines

def plot_combined_latency_vs_queries(df_long, output_dir, baseline_latency=None, smooth_window=5):
    """
    Creates a 1x4 grid of plots, each showing workload latency vs. queries seen
    for one optimizer, styled similarly to the main optimizer grids and with
    non-shared y-axes.
    """
    print("\nGenerating 1x4 combined plot for Latency vs. Queries Seen...")
    df_subset = df_long[
        (df_long['Metric'] == 'Workload Latency (s)') &
        (df_long['Training Policy'] == 'Queries Seen')
    ].copy()

    if df_subset.empty:
        print("No data found for the combined latency vs. queries plot. Skipping.")
        return

    plt.style.use('seaborn-v0_8-whitegrid')
    # Use a consistent color palette, like in the template function
    palette = sns.color_palette("Purples", 6)
    
    # Define a consistent order for the columns
    col_order = sorted(df_long["Training Policy"].unique())
    row_order = sorted(df_long["Metric"].unique())

    # --- Use relplot to create the FacetGrid ---
    # `col='optimizer'` creates the 1x4 layout.
    # `facet_kws={'sharey': False}` is the key for independent y-axes.
    g = sns.relplot(
        data=df_subset,
        x='Progress Value',
        y='Metric Value',
        col="Training Policy",
        row="Metric",
        row_order=row_order,
        col_order=col_order,
        kind='line',
        marker='s',
        color=palette[4], # Use a fixed color for the 'Raw' line
        height=4,
        aspect=1.2,
        facet_kws={'sharey': False, 'sharex': False}, # Independent axes
        legend=False # We will create a custom legend
    )

    # --- Customize each subplot (ax) ---
    for optimizer_name, ax in g.axes_dict.items():
        # Get the specific data for this subplot
        subset = df_subset[df_subset['optimizer'] == optimizer_name]
        if subset.empty:
            continue
        
        subset = subset.sort_values("Progress Value").reset_index(drop=True)

        # Add the smoothed line, just like in the template
        smooth_y = subset["Metric Value"].rolling(window=smooth_window, min_periods=1, center=True).mean()
        ax.plot(subset["Progress Value"], smooth_y,
                color=palette[2], lw=2.5, alpha=0.7, marker="D")

        # Apply log scale, grid, and baseline to each subplot
        ax.set_yscale('log')
        ax.grid(which="major", axis="both", linestyle=":", alpha=0.7)
        if baseline_latency:
            # NEO doesn't get the baseline in other plots, maintain consistency
            if optimizer_name.upper() != "NEO":
                ax.axhline(baseline_latency, ls='--', color='red', lw=2)

    # --- Add figure-level titles, labels, and legend ---
    g.fig.suptitle('Workload Latency vs. Queries Seen Policy', fontsize=18, y=1.08)
    g.set_titles(col_template="{col_name}", size=14)
    g.set_axis_labels("Queries Seen (Progress)", "Workload Latency (s)")

    # Manually create a single, clean legend for the entire figure
    raw_handle = mlines.Line2D([], [], color=palette[4], marker='s', linestyle='-', label='Raw')
    smooth_handle = mlines.Line2D([], [], color=palette[2], marker='D', linestyle='-', label='Smoothed')
    
    legend_handles = [raw_handle, smooth_handle]
    if baseline_latency:
        legend_handles.append(mlines.Line2D([], [], color='red', ls='--', lw=2, label='PostgreSQL Baseline'))

    g.fig.legend(handles=legend_handles, loc='upper center', ncol=3,
                 bbox_to_anchor=(0.5, 1.0), frameon=True)

    plt.subplots_adjust(top=0.85) # Adjust layout to prevent title overlap

    filename = "combined_latency_vs_queries_grid.pdf"
    plt.savefig(Path(output_dir) / filename, bbox_inches='tight')
    plt.show()
    print(f"Saved plot: {filename}")

In [52]:
# --- Configuration ---
MODELS_BASE_DIR = '/data/hdd1/users/kmparmp/Learned-Optimizers-Benchmarking-Suite/models/experiment3'
RESULTS_BASE_DIR = '/data/hdd1/users/kmparmp/Learned-Optimizers-Benchmarking-Suite/experiments/experiment3/test/'
OUTPUT_DIR = '/data/hdd1/users/kmparmp/Learned-Optimizers-Benchmarking-Suite/experiments/experiment3/plots'
OPTIMIZERS = ['BAO', 'LOGER', 'LERO', 'NEO']

os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- Step 1: Consolidate data ---
all_trajectories = []
for optimizer in OPTIMIZERS:
    print(f"\n--- Processing Data for: {optimizer} ---")
    discovered = discover_checkpoints(MODELS_BASE_DIR, optimizer)
    if not discovered:
        print(f"No checkpoints found for {optimizer}. Skipping.")
        continue
    
    df = build_trajectory_df_from_files(discovered, RESULTS_BASE_DIR, optimizer)
    if not df.empty:
        all_trajectories.append(df)

if not all_trajectories:
    print("No data collected for any optimizer. Exiting.")
    exit()

master_df = pd.concat(all_trajectories, ignore_index=True)
pg_baseline_latency = get_postgres_baseline(RESULTS_BASE_DIR)
# MODIFICATION: We still calculate the baseline, just don't plot it for now.
# This makes it easy to re-enable later.
if pg_baseline_latency:
    print(f"\nPostgreSQL Baseline Latency (for reference): {pg_baseline_latency:.2f}s")
else:
    print("\nPostgreSQL baseline results not found.")


# --- Step 2: Reshape data from wide to long format for FacetGrid plotting ---
print("\nReshaping data for master plot...")
dfs_to_concat = []
policy_map = {
    'epoch': ('Epochs', 'epoch'),
    'query': ('Queries Seen', 'queries_seen'),
    'loss': ('Model Loss', 'model_loss')
}
for policy_short, (policy_long, col_name) in policy_map.items():
    df_policy = master_df[master_df['checkpoint_type'] == policy_short].copy()
    # Filter out placeholder values like -1
    df_policy = df_policy[df_policy[col_name] >= 0] 
    
    df_lat = df_policy[['optimizer', col_name, 'cumulative_latency_s']].rename(columns={
        col_name: 'Progress Value', 'cumulative_latency_s': 'Metric Value'
    })
    df_lat['Training Policy'] = policy_long
    df_lat['Metric'] = 'Workload Latency (s)'
    
    df_qerr = df_policy[['optimizer', col_name, 'avg_q_error']].rename(columns={
        col_name: 'Progress Value', 'avg_q_error': 'Metric Value'
    })
    df_qerr['Training Policy'] = policy_long
    df_qerr['Metric'] = 'Average Q-Error'
    
    dfs_to_concat.extend([df_lat, df_qerr])

long_df = pd.concat(dfs_to_concat, ignore_index=True).dropna()
long_df = long_df.sort_values(by=['optimizer', 'Training Policy', 'Progress Value'])


--- Processing Data for: BAO ---


Processing BAO Checkpoints: 100%|██████████| 40/40 [00:01<00:00, 29.30it/s]



--- Processing Data for: LOGER ---


Processing LOGER Checkpoints: 100%|██████████| 47/47 [00:00<00:00, 50.81it/s]



--- Processing Data for: LERO ---


Processing LERO Checkpoints: 100%|██████████| 46/46 [00:00<00:00, 50.15it/s]



--- Processing Data for: NEO ---


Processing NEO Checkpoints: 100%|██████████| 82/82 [00:00<00:00, 671.11it/s]


Searching for Postgres baseline plans in /data/hdd1/users/kmparmp/Learned-Optimizers-Benchmarking-Suite/experiments/experiment3/test/**/classic_qep.json

PostgreSQL Baseline Latency (for reference): 178.98s

Reshaping data for master plot...


In [53]:
# Define what to keep for each optimizer
plot_config = {
    "GLOBAL": [
        ("Workload Latency (s)", "Queries Seen"),
    ],
    "NEO": [
        ("Workload Latency (s)", "Epochs"),
        ("Workload Latency (s)", "Model Loss"),
    ],
    "BAO": [
        ("Workload Latency (s)", "Epochs"),
        ("Average Q-Error", "Epochs"),
        ("Workload Latency (s)", "Model Loss"),
    ],
    "LOGER": [
        ("Workload Latency (s)", "Epochs"),
        ("Average Q-Error", "Epochs"),
        ("Workload Latency (s)", "Queries Seen"),
    ],
    "LERO": [
        ("Workload Latency (s)", "Epochs"),
        ("Workload Latency (s)", "Queries Seen"),
    ]
}

def filter_long_df(df, optimizer, keep_pairs):
    """Filter df for one optimizer based on (Metric, Training Policy) pairs."""
    mask_optimizer = df["optimizer"] == optimizer
    mask_pairs = df.apply(lambda row: (row["Metric"], row["Training Policy"]) in keep_pairs, axis=1)
    return df[mask_optimizer & mask_pairs].copy()


In [ ]:
# neo_df = filter_long_df(long_df, "NEO", plot_config["NEO"])
# plot_optimizer_grid(neo_df, "NEO", OUTPUT_DIR, pg_baseline_latency)

# Example: plot BAO
bao_df = filter_long_df(long_df, "BAO", plot_config["BAO"])
plot_optimizer_grid(bao_df, "BAO", OUTPUT_DIR, pg_baseline_latency)


Generating custom grid plot for NEO...


KeyError: 'Metric'

In [ ]:
# optimizer_name = "BAO"
# df_single_optimizer = long_df[long_df['optimizer'] == optimizer_name]
# if optimizer_name == "LERO":
#     df_single_optimizer = df_single_optimizer[df_single_optimizer['Training Policy'] != "Model Loss"]
# if optimizer_name == "BAO":
#     long_df.loc[(long_df['optimizer'] == 'BAO') & (long_df['Training Policy'] == 'Model Loss'), 'Progress Value'] *= 1000
# if not df_single_optimizer.empty:
#     plot_optimizer_grid(
#         df_single_optimizer,
#         optimizer_name,
#         OUTPUT_DIR,
#         pg_baseline_latency
#     )

**BAO**

- Epoch-based Training:
  - Some signs of overfitting at the end of the curve with the latency regressing
  - The moment the workload latency hits its lowest point, the Q-Error has actually hit a local maximum, suggesting that Q-Error is not the only metric that should be accounted for when tracking learning progress, but rather a component of the tracking
- Queries seen policy actually demonstrates many spikes
- Loss-based training showed the most linear trajectory out of every counterpart, seems to be the best metric for BAO's training module

In [ ]:
# optimizer_name = "LOGER"
# df_single_optimizer = long_df[long_df['optimizer'] == optimizer_name]
# if optimizer_name == "LERO":
#     df_single_optimizer = df_single_optimizer[df_single_optimizer['Training Policy'] != "Model Loss"]    
# if not df_single_optimizer.empty:
#     plot_optimizer_grid(
#         df_single_optimizer,
#         optimizer_name,
#         OUTPUT_DIR,
#         pg_baseline_latency
#     )

**LOGER**

- More susceptible to wide margin regressions during training, across all 3 training paradigms
- Here average Q-Error seems to be a lot better at indicating training progress, as latency local minima map with q-error local minima, and also vice verse about the maxima
- This probably happens because of the Tree-LSTM & GT architecture that structures the embedding space in a better way
- (We still need to find a module that works best for LOGER)

In [ ]:
# optimizer_name = "NEO"
# df_single_optimizer = long_df[long_df['optimizer'] == optimizer_name]
# if optimizer_name == "LERO":
#     df_single_optimizer = df_single_optimizer[df_single_optimizer['Training Policy'] != "Model Loss"]
# if not df_single_optimizer.empty:
#     plot_optimizer_grid(
#         df_single_optimizer,
#         optimizer_name,
#         OUTPUT_DIR,
#         pg_baseline_latency
#     )

**NEO**

- Loss-related checkpoints are very minute (only 15) because the network never went below the 0.93 training loss mark
- So many spikes cause the optimizer does not converge
- The fact that the Model does not converge in terms of training loss explains every behavior we have seen from NEO so far

In [ ]:
# optimizer_name = "LERO"
# df_single_optimizer = long_df[long_df['optimizer'] == optimizer_name]
# if optimizer_name == "LERO":
#     df_single_optimizer = df_single_optimizer[df_single_optimizer['Training Policy'] != "Model Loss"]    
# if not df_single_optimizer.empty:
#     plot_optimizer_grid(
#         df_single_optimizer,
#         optimizer_name,
#         OUTPUT_DIR,
#         pg_baseline_latency
#     )